# Analyzing borrower’s risk of defaulting

Our project is to prepare a report for a bank’s loan division. We’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Our report will be considered when building the **credit score** of a potential customer. The **credit score** is used to evaluate the ability of a potential borrower to repay their loan.


## Open the data file and have a look at the general information. 

In [ ]:
# Loading all the libraries
import pandas as pd
import nltk
nltk.download('all')

from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import warnings

warnings.filterwarnings("ignore")

In [2]:
# Load the data
borrower_data = pd.read_csv('/content/drive/MyDrive/csv_fies/credit_scoring_eng.csv')
borrower_data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


## Task 1. Data exploration

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan


In [3]:
# Let's see how many rows and columns our dataset has 
print('There are', borrower_data.shape[0], 'rows and', borrower_data.shape[1], 'columns in our dataset')

There are 21525 rows and 12 columns in our dataset


In [4]:
# let's print the first N rows
borrower_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


1. In the column `days of employment` there are a lot of negative numbers, which is contrary to common sense.
2. In the `education` column there are different spellings of the same categories - those are duplicates, so we will bring them to lowercase using the `lower()` method and make sure that they are not duplicated.
3. The `family status` and `family id` columns essentially reflect the same information as well as the `education` and `education_id`, so we'll just display `family status` and `education` as a reference table. 
4. In the `purpose` column, we can see similar purposes but phrased differently, we will need to categorize them.

In [5]:
# Get info on data
borrower_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


We can see missing values in the `days_employed` and `total_income` columns.


In [6]:
# Let's look at the filtered table with missing values in the the first column with missing data
borrower_data[borrower_data['days_employed'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


There are 2174 rows with values missing simmetricaly with `days_employed` and `total_income`. Since those two features are related to each other and the ammount of the missing values is the same, the values probably missing because they could be dependant on each other, i.e - a person, who hadn't worked a day never got any income.

In [7]:
# Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.
borrower_filt = borrower_data[(borrower_data['days_employed'].isnull()) & (borrower_data['total_income'].isnull())]
print('Ammount of rows with missing values:', borrower_filt.shape[0])
borrower_filt.head(10)

Ammount of rows with missing values: 2174


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,NaN,transactions with commercial real estate
67,0,NaN,52,bachelor's degree,0,married,0,F,retiree,0,NaN,purchase of the house for my family
72,1,NaN,32,bachelor's degree,0,married,0,M,civil servant,0,NaN,transactions with commercial real estate
82,2,NaN,50,bachelor's degree,0,married,0,F,employee,0,NaN,housing
83,0,NaN,52,secondary education,1,married,0,M,employee,0,NaN,housing


In [8]:
print('The percent of missing data is {:.2%}'.format(len(borrower_filt) / len(borrower_data)))

The percent of missing data is 10.10%


The percent of missing data is so, that a simple remove of the missing data can affect our calculations. We will adress each characteristic, starting with income type and see if there was a correlation.

In [9]:
# Let's investigate clients who do not have data on identified characteristic and the column with the missing values
borrower_filt['income_type'].value_counts()

employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

In [10]:
# Checking distribution
tot_rows = len(borrower_filt)
income_values = borrower_filt['income_type'].value_counts()
distr_income = income_values / tot_rows * 100
distr_income

employee         50.827967
business         23.367065
retiree          18.997240
civil servant     6.761730
entrepreneur      0.045998
Name: income_type, dtype: float64

It seems that information about total income is missing in half of the clients who are employed, and the other half spreaded between the other types of employment, therefore there is no obvious pattern here

In [11]:
# Checking the distribution in the whole dataset
borrower_data['income_type'].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

The distribution of types of employment on the whole dataset is similar to the one we saw on the subset of missing values, so we can conclude that they probably are missing randomly and not depending on an income type.

In [12]:
# Check for other reasons and patterns that could lead to missing values
family_values = borrower_filt['family_status'].value_counts()
distr_family = (family_values / tot_rows) * 100
distr_family

married              56.899724
civil partnership    20.331187
unmarried            13.247470
divorced              5.151794
widow / widower       4.369825
Name: family_status, dtype: float64

In [13]:
education_values = borrower_filt['education'].value_counts()
distr_education = (education_values / tot_rows) * 100
distr_education

secondary education    64.765409
bachelor's degree      22.815087
SECONDARY EDUCATION     3.081877
Secondary Education     2.989880
some college            2.529899
Bachelor's Degree       1.149954
BACHELOR'S DEGREE       1.057958
primary education       0.873965
Some College            0.321987
SOME COLLEGE            0.321987
Primary Education       0.045998
PRIMARY EDUCATION       0.045998
Name: education, dtype: float64

By analysing the distributions we can see the missing values spreaded more or less randomly. Let's have a look at the overall ammount of people who didn't payback the loan to get a general impression.

In [14]:
debt_values = borrower_filt['debt'].value_counts()
distr_debt = (debt_values / tot_rows) * 100
distr_debt

0    92.180313
1     7.819687
Name: debt, dtype: float64

Most of the clients did pay back their loan in time

**Conclusions**


According to the general information about our dataset we noticed two columns with values missing in 10.10% of each of them, after checking the distribution of the missing values on most related charachteristics and the whole dataset no obvious pattern for missing values has occured. The data missing randomly could mean that this happened due to some technical issue (in the process of downloading the data from a server, for example). As the next step we need to take care of those duplicates we saw at the "education" column and see if there are any other problematic values out there.

## Data transformation

Let's go through each column to see what issues we may have in them.

In [15]:
# Let's see all values in education column to check if and what spellings will need to be fixed
borrower_data['education'].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

In [16]:
# Fix the registers if required
borrower_data['education_lowercase'] = borrower_data['education'].str.lower()

In [17]:
# Checking all the values in the column to make sure we fixed them
borrower_data['education_lowercase'].value_counts()

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education_lowercase, dtype: int64

In [18]:
# Let's see the distribution of values in the `children` column
borrower_data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [19]:
borrower_data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Concidering that the ammount of children = 2 is in the top3 most frequent values, we will change the value "20" to "2", assuming that the value "20" occured due to a typo, the same logic goes for negative quantity of children.

In [20]:
# [fix the data based on your decision]
borrower_data['children'] = borrower_data['children'].abs()  # Getting rid of negative values
borrower_data.loc[borrower_data['children'] == 20, 'children'] = 2  # Replacing the value 20 children to 2

In [21]:
# Checking the `children` column again to make sure it's all fixed
borrower_data['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [22]:
# Find problematic data in `days_employed`, if they exist, and calculate the percentage
borrower_data['days_employed'].unique()

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

In [23]:
# Address the problematic values, if they exist
borrower_data['days_employed'] = borrower_data['days_employed'].abs() # Getting rid of negative number

In [24]:
borrower_data['days_employed'] / 24 # Calculating the ammount of days 

0          351.569709
1          167.700156
2          234.309275
3          171.864467
4        14177.753002
             ...     
21520      188.721528
21521    14330.725172
21522       88.056120
21523      129.686738
21524       82.687816
Name: days_employed, Length: 21525, dtype: float64

The absolute most of the data in the `days_employed` column is problematic, for several reasons: 
1. There are negative values which makes no sense in terms of presenting a period of time. 
2. The value presented in the float format which is not needed for the purpose of the project, we don't really need to know the ammount of days with such accuracy 
3. With that being said we will leave it as is for now and will notify our engineers that this problem has occured.

In [25]:
# Check the `dob_years` for suspicious values and count the percentage
borrower_data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [26]:
borrower_data[borrower_data['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase
99,0,346541.618895,0,Secondary Education,1,married,0,F,retiree,0,11406.644,car,secondary education
149,0,2664.273168,0,secondary education,1,divorced,3,F,employee,0,11228.230,housing transactions,secondary education
270,3,1872.663186,0,secondary education,1,married,0,F,employee,0,16346.633,housing renovation,secondary education
578,0,397856.565013,0,secondary education,1,married,0,F,retiree,0,15619.310,construction of own property,secondary education
1040,0,1158.029561,0,bachelor's degree,0,divorced,3,F,business,0,48639.062,to own a car,bachelor's degree
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,NaN,0,secondary education,1,married,0,F,employee,0,NaN,housing,secondary education
20462,0,338734.868540,0,secondary education,1,married,0,F,retiree,0,41471.027,purchase of my own house,secondary education
20577,0,331741.271455,0,secondary education,1,unmarried,4,F,retiree,0,20766.202,property,secondary education
21179,2,108.967042,0,bachelor's degree,0,married,0,M,business,0,38512.321,building a real estate,bachelor's degree


In the age column we have a 0 value age which makes no sence for such a characteristic as 'age', and since it is presented in less then 1% of the dataset we will replace it with an average value.

In [27]:
# Address the issues in the `dob_years` column, if they exist:
# After having checked the mean and median value it seams that they close enough to one another, 
# so there are no strong outliers and we can use median
avg_age = borrower_data['dob_years'].median() 
borrower_data.loc[borrower_data['dob_years'] == 0, 'dob_years'] = avg_age

In [28]:
# Check the result - make sure it's fixed
(borrower_data['dob_years'] != 0).value_counts()

True    21525
Name: dob_years, dtype: int64

In [29]:
# Let's see the values for the column
borrower_data['family_status'].value_counts()


married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

The family status seems proper as it is, so we don't need to change here anything



In [30]:
# Let's see the values in the column
borrower_data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [31]:
# Address the problematic values, if they exist
print(borrower_data.loc[borrower_data['gender'] == 'XNA']) # Altering the index of the row with problenatic value
borrower_data.drop(labels=10701, axis=0, inplace=True)  # Getting rid of the row entirely


       children  days_employed  dob_years     education  education_id  \
10701         0    2358.600502         24  some college             2   

           family_status  family_status_id gender income_type  debt  \
10701  civil partnership                 1    XNA    business     0   

       total_income          purpose education_lowercase  
10701     32624.825  buy real estate        some college  


In [32]:
# Check the result - make sure it's fixed
borrower_data['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

In [33]:
# Let's see the values in the column
borrower_data['income_type'].value_counts()

employee                       11119
business                        5084
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

Now let's see if we have any duplicates in our data. 

In [34]:
# Checking duplicates

print('There are {:} duplicated values in our subset'.format(borrower_data.duplicated().sum()))

There are 55 duplicated values in our subset


In [35]:
# Address the duplicates, if they exist
borrower_data['total_income'].value_counts()

17312.717    2
42413.096    2
31791.384    2
21005.772    1
18591.443    1
            ..
23686.835    1
9606.294     1
28156.762    1
24931.195    1
13127.587    1
Name: total_income, Length: 19347, dtype: int64

Lets try to have a look at the first three duplicates to get an impression of duplicated values

In [36]:
duplicated_income = [17312.717, 31791.384, 42413.096]

In [37]:
for income in duplicated_income:
    display(borrower_data[borrower_data['total_income'] == income])
    print()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase
6335,0,353726.082244,57,secondary education,1,widow / widower,2,F,retiree,0,17312.717,to become educated,secondary education
20529,1,4616.419865,35,BACHELOR'S DEGREE,0,civil partnership,1,M,business,0,17312.717,wedding ceremony,bachelor's degree


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase
10326,1,1723.229736,44,secondary education,1,civil partnership,1,M,employee,0,31791.384,to have a wedding,secondary education
20154,0,1028.200723,51,secondary education,1,married,0,F,employee,0,31791.384,property,secondary education


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase
6614,0,417.482928,48,bachelor's degree,0,married,0,F,business,0,42413.096,getting higher education,bachelor's degree
8377,0,371001.334974,56,secondary education,1,unmarried,4,M,retiree,0,42413.096,real estate transactions,secondary education


Having looked at a sample of duplicated values we can assume that duplicates occured a large ammount of defferent parameters, and concidering that their share in the whole dataset is less then 1% we can safely drop them

In [38]:
borrower_data = borrower_data.drop_duplicates()

In [39]:
# Check the size of the dataset that you now have after your first manipulations with it
print('There are', borrower_data.shape[0], 'rows and', borrower_data.shape[1], 'columns left in our dataset after eliminating the duplicates')
borrower_data.head(10)

There are 21469 rows and 13 columns left in our dataset after eliminating the duplicates


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,bachelor's degree
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,secondary education
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,secondary education
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,secondary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,secondary education
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,bachelor's degree
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,bachelor's degree
7,0,152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education,secondary education
8,2,6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding,bachelor's degree
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,secondary education


Now that we got rid of little problems, such as negative numbers and non-logical values in data and treated duplicates we will move on to dealing with missing values

# Working with missing values

We have formed two dictionaries - of education and family status keys and values, in order to ease operating with different kinds of statuses, for example: a merried clients' status is 0. But we will not use them for this project.

In [40]:
# Find the dictionaries
education_dict = {'0' : "bachelor's degree", 
                  '1' : 'secondary education', 
                  '2' : 'some college', 
                  '3' : 'primary education', 
                  '4' : "bachelor's degree"}

marital_stat_dict = {'0' : 'married', 
                     '1' : 'civil partnership', 
                     '2' : 'widow / widower',  
                     '3' : 'divorced', 
                     '4' : 'unmarried'}

### Restoring missing values in `total_income`

There are two culumns where we meet missing values, but only one of them we will treat - the `total income`.

In [41]:
# Let's write a function that calculates the age category
def age_category(age):
    if age <= 18.0:
        return 'minors'
    if age <= 64.0:
        return 'adult'
    return 'retired'    

In [42]:
# Test if the function works
print(age_category(17.0))
print(age_category(64.0))
print(age_category(75.0))

minors
adult
retired


In [43]:
# Creating new column based on function
borrower_data['category_by_age'] = borrower_data['dob_years'].apply(age_category)

In [44]:
# Checking how values in the new column
borrower_data['category_by_age'].value_counts()

adult      20571
retired      898
Name: category_by_age, dtype: int64

Create a table that only has data without missing values. This data will be used to restore the missing values.

In [45]:
# Create a table without missing values and print a few of its rows to make sure it looks fine
borrower_clean = borrower_data[(borrower_data.days_employed.notna())&(borrower_data.total_income.notna())]
borrower_clean

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase,category_by_age
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,bachelor's degree,adult
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,secondary education,adult
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,secondary education,adult
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,secondary education,adult
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,secondary education,adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,secondary education,adult
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,secondary education,retired
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,secondary education,adult
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,secondary education,adult


In [46]:
# Look at the mean values for income based on your identified factors
avg_income = borrower_clean['total_income'].mean()
print('The mean value of total income is: {:.2f}'.format(avg_income))

The mean value of total income is: 26787.27


In [47]:
# Look at the median values for income based on your identified factors
median_income = borrower_clean['total_income'].median()
print('The median of the total income is: {:.2f}'.format(median_income))

The median of the total income is: 23201.87


In [48]:
avg_age = borrower_clean['dob_years'].mean()
print('The avarage age is: {:.2f}'.format(avg_age))

The avarage age is: 43.45


In [49]:
median_age = borrower_clean['dob_years'].median()
print('The median age is: {:.0f}'.format(median_age))

The median age is: 42


In [50]:
pivot = borrower_clean.pivot_table(
index = 'category_by_age',
columns = 'income_type',
values = 'days_employed',
aggfunc = 'mean')
pivot

income_type,business,civil servant,employee,entrepreneur,paternity / maternity leave,retiree,student,unemployed
category_by_age,,,,,,,,
adult,2101.315980,3387.057925,2313.993758,520.848083,3296.759962,364791.529478,578.751554,366413.652744
retired,3292.768472,4377.924812,4366.938943,NaN,NaN,365832.591336,NaN,NaN


We will use the median for both age and income filling values because it is a more precise number.

In [51]:
#  Write a function that we will use for filling in missing values
median_by_income = borrower_clean.groupby('income_type')['total_income'].median()
median_by_income

income_type
business                       27571.0825
civil servant                  24071.6695
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        18962.3180
student                        15712.2600
unemployed                     21014.3605
Name: total_income, dtype: float64

In [52]:
borrower_data['total_income'] = borrower_data.groupby(['income_type'])['total_income']\
                                            .transform(lambda x: x.fillna(x.median()))

In [53]:
# Check if it works
borrower_data[borrower_data['total_income'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase,category_by_age


In [54]:
# Checking the number of entries in the columns
borrower_data['total_income'].count()

21469

###  Restoring values in `days_employed`

In this project we will replace missing values in the `days_employed` with the median as well, but we will not be using this data at all because it had a lot of negative numbers that we don't know how occured and as we previously stated this data is problematic to take it into analisys.

In [55]:
# Distribution of `days_employed` medians based on your identified parameters
median_days_by_income = borrower_clean.groupby('income_type')['days_employed'].median()
median_days_by_income


income_type
business                         1546.333214
civil servant                    2689.368353
employee                         1574.202821
entrepreneur                      520.848083
paternity / maternity leave      3296.759962
retiree                        365213.306266
student                           578.751554
unemployed                     366413.652744
Name: days_employed, dtype: float64

In [56]:
# Distribution of `days_employed` means based on your identified parameters
mean_days_by_income = borrower_clean.groupby('income_type')['days_employed'].mean()
mean_days_by_income


income_type
business                         2111.470404
civil servant                    3399.896902
employee                         2326.499216
entrepreneur                      520.848083
paternity / maternity leave      3296.759962
retiree                        365003.491245
student                           578.751554
unemployed                     366413.652744
Name: days_employed, dtype: float64

In [57]:
borrower_data['days_employed'].describe()

count     19350.000000
mean      66918.065141
std      139033.698578
min          24.141633
25%         926.990457
50%        2194.218768
75%        5538.423086
max      401755.400475
Name: days_employed, dtype: float64

Medians are more safe to use when we have outliers and we certeinly do have them - the minimum and maximum values of `days_employed` are in rather a big range.

In [58]:
# Let's write a function that calculates means or medians (depending on your decision) based on your identified parameter
borrower_data['days_employed'] = borrower_data.groupby('income_type')['days_employed']\
                                                .transform(lambda x: x.fillna(x.median()))

In [59]:
# Check that the function works
borrower_data[borrower_data['days_employed'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase,category_by_age


[When you think you've finished with `total_income`, check that the total number of values in this column matches the number of values in other ones.]

In [60]:
# Check the entries in all columns - make sure we fixed all missing values
borrower_data.isnull().sum()


children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
education_lowercase    0
category_by_age        0
dtype: int64

## Categorization of data

In [61]:
# Print the values for your selected data for categorization
borrower_data['purpose'].describe()

count                21469
unique                  38
top       wedding ceremony
freq                   793
Name: purpose, dtype: object

Let's check unique values

In [62]:
# Check the unique values
borrower_data['purpose'].unique() 


array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [63]:
wordnet_lemma = WordNetLemmatizer()

In [67]:
phrase = 'purchase of the house'
token = nltk.word_tokenize(phrase)
print(token)
lemmas = [wordnet_lemma.lemmatize(t, pos = 'n') for t in token]
print(lemmas)

['purchase', 'of', 'the', 'house']
['purchase', 'of', 'the', 'house']


In [68]:
lemmas_list = []

for purpose in borrower_data['purpose'].unique():
    words = nltk.word_tokenize(purpose)
    lemmas = [wordnet_lemma.lemmatize(w, pos = 'n') for w in words]
    for i in lemmas:
        lemmas_list.append(i)
        
lemmas_list

['purchase',
 'of',
 'the',
 'house',
 'car',
 'purchase',
 'supplementary',
 'education',
 'to',
 'have',
 'a',
 'wedding',
 'housing',
 'transaction',
 'education',
 'having',
 'a',
 'wedding',
 'purchase',
 'of',
 'the',
 'house',
 'for',
 'my',
 'family',
 'buy',
 'real',
 'estate',
 'buy',
 'commercial',
 'real',
 'estate',
 'buy',
 'residential',
 'real',
 'estate',
 'construction',
 'of',
 'own',
 'property',
 'property',
 'building',
 'a',
 'property',
 'buying',
 'a',
 'second-hand',
 'car',
 'buying',
 'my',
 'own',
 'car',
 'transaction',
 'with',
 'commercial',
 'real',
 'estate',
 'building',
 'a',
 'real',
 'estate',
 'housing',
 'transaction',
 'with',
 'my',
 'real',
 'estate',
 'car',
 'to',
 'become',
 'educated',
 'second-hand',
 'car',
 'purchase',
 'getting',
 'an',
 'education',
 'car',
 'wedding',
 'ceremony',
 'to',
 'get',
 'a',
 'supplementary',
 'education',
 'purchase',
 'of',
 'my',
 'own',
 'house',
 'real',
 'estate',
 'transaction',
 'getting',
 'higher'

In [69]:
from collections import Counter

Counter(lemmas_list)

Counter({'a': 9,
         'an': 1,
         'become': 1,
         'building': 2,
         'buy': 4,
         'buying': 3,
         'car': 9,
         'ceremony': 1,
         'commercial': 2,
         'construction': 1,
         'educated': 1,
         'education': 7,
         'estate': 7,
         'family': 1,
         'for': 2,
         'get': 1,
         'getting': 2,
         'going': 1,
         'have': 1,
         'having': 1,
         'higher': 1,
         'house': 3,
         'housing': 3,
         'my': 4,
         'of': 5,
         'out': 1,
         'own': 4,
         'profile': 1,
         'property': 4,
         'purchase': 6,
         'real': 7,
         'renovation': 1,
         'renting': 1,
         'residential': 1,
         'second-hand': 2,
         'supplementary': 2,
         'the': 2,
         'to': 6,
         'transaction': 4,
         'university': 2,
         'wedding': 3,
         'with': 2})

By having looked at the list of our keywords we can state some new cregories for our data:

In [70]:
# Let's write a function to categorize the data based on common topics
education_phrases = ['education', 'educated', 'university']


In [71]:
real_estate_phrases = ['house', 'housing', 'real', 'estate', 
                       'commercial','residential', 'construction', 
                       'property', 'building', 'renting', 'renovation']

In [72]:
wedding_phrases = ['wedding', 'ceremony']

In [73]:
vehicle_phrases = ['car','cars']

Lets start our function by testing it just on one phrase

In [74]:
def purpose_category(purpose):
    category=[]
    words = nltk.word_tokenize(purpose)
    lemmas = [wordnet_lemma.lemmatize(w, pos = 'n') for w in words]
    for word in lemmas: 
        return lemmas

In [75]:
test = borrower_data.loc[0]['purpose']
purpose_category(test)

['purchase', 'of', 'the', 'house']

Now we can finish composing our function

In [76]:
def purpose_category(purpose):
    category=[]
    words = nltk.word_tokenize(purpose)
    lemmas = [wordnet_lemma.lemmatize(w, pos = 'n') for w in words] 
    if any(word in lemmas for word in education_phrases):
        return 'education'
    elif  any(word in lemmas for word in real_estate_phrases):
        return 'real estate expenses'
    elif  any(word in lemmas for word in wedding_phrases):
        return 'wedding expenses'
    elif  any(word in lemmas for word in vehicle_phrases):
        return 'vehicle purchase'


In [77]:
# Create a column with the categories and count the values for them
borrower_data['category_by_purpose'] = borrower_data['purpose'].apply(purpose_category)

In [78]:
borrower_data['category_by_purpose'].value_counts()

real estate expenses    10813
vehicle purchase         4308
education                4014
wedding expenses         2334
Name: category_by_purpose, dtype: int64

In [79]:
# Looking through all the numerical data in your selected column for categorization
borrower_data['total_income']

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21520    35966.698
21521    24959.969
21522    14347.610
21523    39054.888
21524    13127.587
Name: total_income, Length: 21469, dtype: float64

In [80]:
# Getting summary statistics for the column
borrower_data['total_income'].describe()

count     21469.000000
mean      26447.078473
std       15705.198189
min        3306.762000
25%       17224.099000
50%       22815.103500
75%       31320.429000
max      362496.645000
Name: total_income, dtype: float64

By having a look at the statistic charachteristics in the `total income` we can recognize approximate levels as per the quartiles.

In [81]:
# Creating function for categorizing into different numerical groups based on ranges
def income_category(income):
    if income < 15000:
        return 'low level'
    if 15001 < income < 25000:
        return 'middle level'
    if 25001 < income < 30000:
        return 'above middle level'
    elif income > 30001:
        return 'high level'

In [82]:
# Creating column with categories
borrower_data['category_by_income'] = borrower_data['total_income'].apply(income_category)

In [83]:
# Count each categories values to see the distribution
borrower_data['category_by_income'].value_counts()

middle level          8618
high level            5919
low level             3743
above middle level    3185
Name: category_by_income, dtype: int64

## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

In [84]:
# Check the children data and paying back on time
print(borrower_data.groupby('children')['debt'].value_counts())

# Calculating default-rate based on the number of children
borrower_data.groupby(['children'])['debt'].mean().reset_index().sort_values(by='debt')


children  debt
0         0       13042
          1        1063
1         0        4411
          1         445
2         0        1926
          1         202
3         0         303
          1          27
4         0          37
          1           4
5         0           9
Name: debt, dtype: int64


,children,debt
5,5,0.000000
0,0,0.075363
3,3,0.081818
1,1,0.091639
2,2,0.094925
4,4,0.097561


**Conclusion**
We can see that all the clients with 5 children have paid back their loan.

**Is there a correlation between family status and paying back on time?**

In [85]:
# Check the family status data and paying back on time
print(borrower_data.groupby('family_status')['debt'].value_counts())


# Calculating default-rate based on family status
borrower_data.groupby(['family_status'])['debt'].mean().reset_index().sort_values(by='debt')


family_status      debt
civil partnership  0        3773
                   1         388
divorced           0        1110
                   1          85
married            0       11413
                   1         931
unmarried          0        2536
                   1         274
widow / widower    0         896
                   1          63
Name: debt, dtype: int64


,family_status,debt
4,widow / widower,0.065693
1,divorced,0.071130
2,married,0.075421
0,civil partnership,0.093247
3,unmarried,0.097509


**Conclusion**

We can say that the widowers paid back least acuurately then other clients, while the unmerried is the category has the highets value.

**Is there a correlation between income level and paying back on time?**

In [86]:
print('In average {:.0%} of clients did not repay their loan'.format(borrower_data['debt'].mean()))

In average 8% of clients did not repay their loan


In [87]:
# Check the income level data and paying back on time
print(borrower_data.groupby('category_by_income')['debt'].value_counts())


# Calculating default-rate based on income level
borrower_data.groupby(['category_by_income'])['debt'].mean().reset_index().sort_values(by='debt')


category_by_income  debt
above middle level  0       2916
                    1        269
high level          0       5483
                    1        436
low level           0       3445
                    1        298
middle level        0       7880
                    1        738
Name: debt, dtype: int64


,category_by_income,debt
1,high level,0.073661
2,low level,0.079615
0,above middle level,0.084458
3,middle level,0.085635


**Conclusion**

The percentage depending on income level seems to be distributed evenly.

**How does credit purpose affect the default rate?**

In [88]:
# Check the percentages for default rate for each credit purpose and analyze them
borrower_data.groupby(['category_by_purpose'])['debt'].mean().reset_index().sort_values(by='debt')


,category_by_purpose,debt
1,real estate expenses,0.072320
3,wedding expenses,0.079692
0,education,0.092177
2,vehicle purchase,0.093547


**Conclusion**

All clients with real estate expenses paid their loans in a more accurate way than clients with other credit purposes, clients who used their loans to purchase a vehicle have the most ammount of debt.

# General Conclusion 


In our dataset we have come across with some problematic data: 10.10% of our data was missing randomly in columns which refer to ammount of the clients days employed and their total income simmetrically, the missing values were filled with a median value for each column. We treated the duplicates in the education column manually and dropped all the duplicated rows as their share in the whole dataset was less then 1%. We had a problematic value in the information about the age (0 age value) which we also rplaced with a median value. To have a better view in compering data with each other we categorized the age, total income and purpose columns

After checking the chosen charachteristics we couldn't clearly define that there is a strong correlation between paying a debt on time and having a certein purpose for the loan or a certein family status or a particular level of income. However, we did see that all clients with ammount of children equaling to 5 did pay their loan on time, so as most of the widowers, and clients whos' purpose of credit was real estate expenses. And to the opposite: most of the clients who purchased cars and unmerried clients didn't pay their loan on time.